# Analysis

This notebook presents execution and results from:

* Base case analysis
* Scenario analysis
* Sensitivity analysis

The run time is provided at the end of the notebook.

Credit:

* Analysis of the spread of replication results was adapted from Tom Monks (2024) HPDM097 - Making a difference with health data (https://github.com/health-data-science-OR/stochastic_systems) (MIT License).

## Set-up

Load `.ipynb` linters, and load required packages.

In [1]:
%load_ext pycodestyle_magic

In [2]:
%pycodestyle_on

In [3]:
import itertools
import pandas as pd
import plotly.express as px
import time

from simulation.logging import Sim_Logger
from simulation.model import Defaults, Model, Trial, summary_stats

Start timer.

In [4]:
start_time = time.time()

## Spread of replication results

In [5]:
param = Defaults()
trial = Trial(param)
trial.run_trial()

In [6]:
def plot_results_spread(column, x_label, y_label='Frequency'):
    """
    Plot spread of results from across replications, for chosen column.

    Arguments:
        column (str):
            Name of column to plot.
        x_label (str):
            X axis label.
        y_label (str):
            Y axis label
    """
    fig = px.histogram(trial.trial_results_df[column])
    fig.update_layout(
        xaxis_title=x_label,
        yaxis_title=y_label
    )
    fig.show()

In [7]:
plot_results_spread('arrivals', 'Arrivals')
plot_results_spread('mean_q_time_nurse', 'Mean wait time for nurse')
plot_results_spread('mean_time_with_nurse',
                    'Mean length of nurse consultation')
plot_results_spread('mean_nurse_utilisation', 'Mean nurse utilisation')

## Scenario analysis

In [8]:
def run_scenarios(scenarios):
    """
    Run a set of scenarios and return the scenario-level results.

    Arguments:
        scenarios (dict):
            Dictionary where key is name of parameter and value is a list
            with different values to run in scenarios
    """
    # Find every possible permutation of the scenarios
    all_scenarios_tuples = list(itertools.product(*scenarios.values()))
    # Convert back into dictionaries
    all_scenarios_dicts = [
        dict(zip(scenarios.keys(), p)) for p in all_scenarios_tuples]
    # Preview some of the scenarios
    print(f'There are {len(all_scenarios_dicts)} scenarios. Running:')

    # Run the scenarios...
    results = []
    for index, scenario_to_run in enumerate(all_scenarios_dicts):
        print(scenario_to_run)

        # Overwrite defaults from the passed dictionary
        param = Defaults()
        param.scenario_name = index
        for key in scenario_to_run:
            setattr(param, key, scenario_to_run[key])

        # Run trial and keep trial-level results, adding the scenario values to
        # the results dataframe
        scenario_trial = Trial(param)
        scenario_trial.run_trial()
        for key in scenario_to_run:
            scenario_trial.trial_results_df[key] = scenario_to_run[key]
        results.append(scenario_trial.trial_results_df)
    return pd.concat(results)

In [9]:
# Run scenarios
scenario_results = run_scenarios({
    'patient_inter': [3, 4, 5, 6, 7],
    'number_of_nurses': [5, 6, 7, 8]
    })

There are 20 scenarios. Running:
{'patient_inter': 3, 'number_of_nurses': 5}
{'patient_inter': 3, 'number_of_nurses': 6}
{'patient_inter': 3, 'number_of_nurses': 7}
{'patient_inter': 3, 'number_of_nurses': 8}
{'patient_inter': 4, 'number_of_nurses': 5}
{'patient_inter': 4, 'number_of_nurses': 6}
{'patient_inter': 4, 'number_of_nurses': 7}
{'patient_inter': 4, 'number_of_nurses': 8}
{'patient_inter': 5, 'number_of_nurses': 5}
{'patient_inter': 5, 'number_of_nurses': 6}
{'patient_inter': 5, 'number_of_nurses': 7}
{'patient_inter': 5, 'number_of_nurses': 8}
{'patient_inter': 6, 'number_of_nurses': 5}
{'patient_inter': 6, 'number_of_nurses': 6}
{'patient_inter': 6, 'number_of_nurses': 7}
{'patient_inter': 6, 'number_of_nurses': 8}
{'patient_inter': 7, 'number_of_nurses': 5}
{'patient_inter': 7, 'number_of_nurses': 6}
{'patient_inter': 7, 'number_of_nurses': 7}
{'patient_inter': 7, 'number_of_nurses': 8}


Example plot

In [10]:
def plot_scenario(results, x_var, result_var, colour_var, xaxis_title,
                  yaxis_title, legend_title):
    """
    Plot results from difference model scenarios.

    Arguments:
        results (pd.DataFrame):
            Contains results to plot.
        x_var (str):
            Name of variable to plot on X axis.
        result_var (str):
            Name of variable with results, to plot on Y axis.
        colour_var (str|None):
            Name of variable to colour lines with (or set to None).
        xaxis_title (str):
            Title for x axis
        yaxis_title (str):
            Title for y axis
        legend_title (str):
            Title for figure legend
    """
    # If x_var and colour_var are provided, combine both in a list to use
    # as grouping variables when calculating average results
    if colour_var is not None:
        group_vars = [x_var, colour_var]
    else:
        group_vars = [x_var]

    # Calculate average results from each scenario
    df = (
        results
        .groupby(group_vars)[result_var]
        .apply(summary_stats)
        .apply(pd.Series)
        .reset_index()
    )
    df.columns = (list(df.columns[:-4]) +
                  ['mean', 'std_dev', 'ci_lower', 'ci_upper'])

    # Plot mean line
    fig = px.line(df, x=x_var, y='mean', color=colour_var)
    fig.update_layout(
        xaxis_title=xaxis_title,
        yaxis_title=yaxis_title,
        legend_title_text=legend_title
    )

    # Plot confidence interval lines
    for ci in ['ci_upper', 'ci_lower']:
        trace = (px.line(df, x=x_var, y=ci, color=colour_var)
                 .update_traces(opacity=0.5, showlegend=False)
                 .select_traces())
        # Add to figure
        fig.add_traces(list(trace))
    fig.show()

    return df, fig

In [11]:
result, fig = plot_scenario(
    results=scenario_results,
    x_var='patient_inter',
    result_var='mean_q_time_nurse',
    colour_var='number_of_nurses',
    xaxis_title='Patient inter-arrival time',
    yaxis_title='Mean wait time for nurse (minutes)',
    legend_title='Nurses')

In [12]:
result, fig = plot_scenario(
    results=scenario_results,
    x_var='patient_inter',
    result_var='mean_nurse_utilisation',
    colour_var='number_of_nurses',
    xaxis_title='Patient inter-arrival time',
    yaxis_title='Mean nurse utilisation',
    legend_title='Nurses')

Example table

In [13]:
table = result.copy()

# Combine mean and CI into single column, and round
table['mean_ci'] = table.apply(
    lambda row:
    f'{row['mean']:.2f} ({row['ci_lower']:.2f}, {row['ci_upper']:.2f})', axis=1
)

# Convert from long to wide format
table = (
    table
    .pivot(index='patient_inter', columns='number_of_nurses', values='mean_ci')
    .rename_axis('Patient inter-arrival time', axis='index')
    .rename_axis('Number of nurses', axis='columns')
)

# Convert to latex
print(table.to_latex())

\begin{tabular}{lllll}
\toprule
Number of nurses & 5 & 6 & 7 & 8 \\
Patient inter-arrival time &  &  &  &  \\
\midrule
3 & 0.66 (0.66, 0.67) & 0.55 (0.55, 0.56) & 0.47 (0.47, 0.48) & 0.41 (0.41, 0.42) \\
4 & 0.50 (0.49, 0.50) & 0.41 (0.41, 0.42) & 0.36 (0.35, 0.36) & 0.31 (0.31, 0.31) \\
5 & 0.40 (0.40, 0.40) & 0.33 (0.33, 0.34) & 0.29 (0.28, 0.29) & 0.25 (0.25, 0.25) \\
6 & 0.33 (0.33, 0.33) & 0.28 (0.28, 0.28) & 0.24 (0.24, 0.24) & 0.21 (0.21, 0.21) \\
7 & 0.29 (0.28, 0.29) & 0.24 (0.24, 0.24) & 0.20 (0.20, 0.21) & 0.18 (0.18, 0.18) \\
\bottomrule
\end{tabular}



## Sensitivity analysis

Can use similar code to perform sensitivity analyses.

**How does sensitivity analysis differ from scenario analysis?**

* Scenario analysis focuses on a set of predefined situations which are plausible or relevant to the problem being studied. It can often involve varying multiple parameters simulatenously. The purpose is to understand how the system operates under different hypothetical scenarios.
* Sensitivity analysis varies one (or a small group) of parameters and assesses the impact of small changes in that parameter on outcomes. The purpose is to understand how uncertainty in the inputs affects the model, and how robust results are to variation in those inputs.

In [14]:
# Run scenarios
sensitivity_consult = run_scenarios({
    'mean_n_consult_time': [8, 9, 10, 11, 12, 13, 14, 15]
})

There are 8 scenarios. Running:
{'mean_n_consult_time': 8}
{'mean_n_consult_time': 9}
{'mean_n_consult_time': 10}
{'mean_n_consult_time': 11}
{'mean_n_consult_time': 12}
{'mean_n_consult_time': 13}
{'mean_n_consult_time': 14}
{'mean_n_consult_time': 15}


In [15]:
result, fig = plot_scenario(
    results=sensitivity_consult,
    x_var='mean_n_consult_time',
    result_var='mean_q_time_nurse',
    colour_var=None,
    xaxis_title='Mean nurse consultation time (minutes)',
    yaxis_title='Mean wait time for nurse (minutes)',
    legend_title='Nurses'
)

In [16]:
table = result.copy()

# Combine mean and CI into single column, and round
table['mean_ci'] = table.apply(
    lambda row:
    f'{row['mean']:.2f} ({row['ci_lower']:.2f}, {row['ci_upper']:.2f})', axis=1
)

# Select relevant columns and rename
cols = {
    'mean_n_consult_time': 'Mean nurse consultation time',
    'mean_ci': 'Mean wait time for nurse (95% confidence interval)'
}
table = table[cols.keys()].rename(columns=cols)

# Convert to latex
print(table.to_latex())


18:1: W391 blank line at end of file


\begin{tabular}{lrl}
\toprule
 & Mean nurse consultation time & Mean wait time for nurse (95% confidence interval) \\
\midrule
0 & 8 & 0.15 (0.14, 0.16) \\
1 & 9 & 0.28 (0.27, 0.30) \\
2 & 10 & 0.50 (0.47, 0.52) \\
3 & 11 & 0.84 (0.80, 0.88) \\
4 & 12 & 1.36 (1.29, 1.42) \\
5 & 13 & 2.15 (2.05, 2.24) \\
6 & 14 & 3.37 (3.22, 3.52) \\
7 & 15 & 5.30 (5.06, 5.54) \\
\bottomrule
\end{tabular}



## NaN results

Note: In this model, if patients are still waiting to be seen at the end of the simulation, they will have NaN results.

In [17]:
param = Defaults()
param.patient_inter = 2
trial = Trial(param)
trial.run_trial()
trial.patient_results_df.tail()

,patient_id,arrival_time,q_time_nurse,time_with_nurse,run
667686,21586,61913.030043,NaN,NaN,30
667687,21587,61915.384561,NaN,NaN,30
667688,21588,61915.421934,NaN,NaN,30
667689,21589,61917.817910,NaN,NaN,30
667690,21590,61919.845349,NaN,NaN,30


## Example run with logs

The `Sim_Logger` class is used to log events during the simulation. These can be print to the console (`log_to_console`) or saved to a file (`log_to_file`).

This will output lots of information to the screen - currently set to give information on each patient as they arrive and then see the nurse. Therefore, it is only best used when running the simulation for a short time with few patients.

The logs in `model.py` can be altered to print your desired information during the simulation run, which can be helpful during development.

In [18]:
# Mini run of simulation with logger enabled
param = Defaults()
param.warm_up_period = 50
param.data_collection_period = 100
param.number_of_runs = 1
param.cores = 0
param.logger = Sim_Logger(log_to_console=True, log_to_file=False)

model = Model(param, run_number=0)
model.run()

2025-01-08 16:15:21,807 - INFO - logging.py:log():128 - Initialised model: {'param': <simulation.model.Defaults object at 0x7de2e077bac0>, 'run_number': 0, 'env': <simpy.core.Environment object at 0x7de2e0829d30>, 'nurse': <simpy.resources.resource.Resource object at 0x7de2e0829e80>, 'patients': [], 'nurse_time_used': 0, 'nurse_consult_count': 0, 'running_mean_nurse_wait': 0, 'audit_list': [], 'results_list': [], 'patient_inter_arrival_dist': <simulation.model.Exponential object at 0x7de2e0829fd0>, 'nurse_consult_time_dist': <simulation.model.Exponential object at 0x7de2e0820e10>}
2025-01-08 16:15:21,811 - INFO - logging.py:log():128 - Parameters: {'_initialising': False, 'patient_inter': 4, 'mean_n_consult_time': 10, 'number_of_nurses': 5, 'warm_up_period': 50, 'data_collection_period': 100, 'number_of_runs': 1, 'audit_interval': 120, 'scenario_name': 0, 'cores': 0, 'logger': <simulation.logging.Sim_Logger object at 0x7de2e0816eb0>}
2025-01-08 16:15:21,812 - INFO - logging.py:log():12

This will align with the recorded results of each patient (though we only save those that arrive after the warm-up period).

In [19]:
# Compare to patient-level results
model.results_list

[{'patient_id': 1,
  'arrival_time': 51.90400587259546,
  'q_time_nurse': 0.0,
  'time_with_nurse': 18.07891954142075},
 {'patient_id': 2,
  'arrival_time': 51.963434706622714,
  'q_time_nurse': 0.0,
  'time_with_nurse': 3.1020092355006064},
 {'patient_id': 3,
  'arrival_time': 74.3494580155259,
  'q_time_nurse': 0.0,
  'time_with_nurse': 26.744513862017026},
 {'patient_id': 4,
  'arrival_time': 77.53382703300574,
  'q_time_nurse': 0.0,
  'time_with_nurse': 0.7481033661053572},
 {'patient_id': 5,
  'arrival_time': 78.93233230430721,
  'q_time_nurse': 0.0,
  'time_with_nurse': 0.5277574384602378},
 {'patient_id': 6,
  'arrival_time': 86.81473043550623,
  'q_time_nurse': 0.0,
  'time_with_nurse': 2.4349563515001904},
 {'patient_id': 7,
  'arrival_time': 89.78326290873765,
  'q_time_nurse': 0.0,
  'time_with_nurse': 9.665598479334754},
 {'patient_id': 8,
  'arrival_time': 89.80720556833339,
  'q_time_nurse': 0.0,
  'time_with_nurse': 7.004542644523265},
 {'patient_id': 9,
  'arrival_time'

## Run time

In [20]:
# Get run time in seconds
end_time = time.time()
runtime = round(end_time - start_time)

# Display converted to minutes and seconds
print(f'Notebook run time: {runtime // 60}m {runtime % 60}s')

Notebook run time: 0m 54s
